In [6]:
import json
import random
import numpy as np
import sympy as sp
from sympy import sympify, Function, dsolve, Eq, Derivative, sin, cos, symbols, Symbol,lambdify
from sympy.abc import x

def load_expressions(filepaths):
    lines = []
    for filepath in filepaths:
        fin = open(filepath, 'r')
        lines.extend(fin.readlines())
        fin.close()
    #
    random.shuffle(lines)
    exprs = []
    for line in lines:
        data = json.loads(line)
        for k,v in data.items():
            if k in ('f_t', 'g_t'):
                expr = sympify(v[1])
                exprs.append(expr)
    return exprs

exprs = load_expressions(['datasets/parametric_equations.json'])  #'datasets/function_evaluation.json', 

In [7]:
f = sympify('t**2/3+2*t+3.3333')
f

t**2/3 + 2*t + 3.3333

In [2]:
from sympy.calculus.util import continuous_domain, function_range
from sympy import Interval, S

DEFAULT_LEFT = -4.0
DEFAULT_RIGHT = 4.0

def get_data_points(f):
    t = Symbol('t')
    #display(f)
    fl = lambdify((t), f, 'numpy')
    dom = continuous_domain(f, t, S.Reals)
    intv = dom.intersect(Interval(DEFAULT_LEFT, DEFAULT_RIGHT))
    #print("interval:", intv)
    ts = np.arange(float(intv.left), float(intv.right), 0.001)
    ys = fl(ts)
    return ts, ys

In [3]:
import torch
import os, sys
import symbolicregression
from symbolicregression import model as symbolicregression_model
import sympytorch
import requests

model_path = "ckpt/model.pt" 
try:
    if not os.path.isfile(model_path): 
        print("Downloading model...")
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        model = torch.load(model_path)
        model = model.cuda()
    print(model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)    
    
est = symbolicregression_model.SymbolicTransformerRegressor(
                        model=model,
                        max_input_points=10001,
                        n_trees_to_refine=5,
                        rescale=True
                        )

cuda:0
Model successfully loaded!


In [4]:
# import errno
# import os
# import signal
# import functools

# class TimeoutError(Exception):
#     pass

# def timeout(seconds=15, error_message=os.strerror(errno.ETIME)):
#     def decorator(func):
#         def _handle_timeout(signum, frame):
#             raise TimeoutError(error_message)

#         @functools.wraps(func)
#         def wrapper(*args, **kwargs):
#             signal.signal(signal.SIGALRM, _handle_timeout)
#             signal.alarm(seconds)
#             try:
#                 result = func(*args, **kwargs)
#             finally:
#                 signal.alarm(0)
#             return result

#         return wrapper

#     return decorator

In [5]:
from utils.utils import 
from sympy.core.rules import Transform

@timeout(15)
def symbolicregression(f, verbose=True, num_digits=2):
    display(f)
    ts, ys = get_data_points(f)
    if np.isnan(ys).any():
        print("Has NaN")
    ts = np.reshape(ts, (len(ts),1))
    ys = np.reshape(ys, (len(ts),1))
    #
    est.fit(ts,ys)
    replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
    model_str = est.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
    for op,replace_op in replace_ops.items():
        model_str = model_str.replace(op,replace_op)
    #
    x_0,t = symbols(['x_0', 't'])
    raw_expr = sp.parse_expr(model_str)
    expr = sp.expand(raw_expr)
    rounded_expr = expr.xreplace(Transform(lambda x: x.round(num_digits), lambda x: isinstance(x, sp.Float)))
    rounded_expr = rounded_expr.subs(x_0, t)
    display(rounded_expr)
    return expr, rounded_expr

seen_exprs = set()
fin = open("datasets/parametric_equations_sbr.json", 'r')
lines = fin.readlines()
for line in lines:
    data = json.loads(line)
    seen_exprs.add(data['original'])
fin.close()

fout = open("datasets/parametric_equations_sbr.json", 'a')
for i in range(len(exprs)):
    f = exprs[i]
    if str(f) in seen_exprs:
        continue
    try:
        expr, rounded_expr = symbolicregression(f)
        result = {'original':str(f), 'regressed':str(expr), 'rounded_regressed':str(rounded_expr)}
        fout.write(json.dumps(result)+'\n')
        fout.flush()
    except:
        continue

fout.close()

13*t/3 + 26/3

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


4.33*t + 8.67

8*t**2 - 240*t + 1797

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


...

276.39*t**2 + 1529.5*t + 2116.0

2116*(23*t**2 - 210*t + 476)**2/2401

64*t/3 + 112

(32*t**2 + 240*t + 447)**2


KeyboardInterrupt

